In [1]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from sentence_transformers import SentenceTransformer, util

In [4]:
# Load your own model
model_path = '../project3/src/apple_resnet_classifier.pt'
#'C:/Users/jiyoo/workspace/MakeAIWork3/project3/src/apple_resnet_classifier.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(model_path, map_location=device)
model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# Define the image transformations
transform_img_normal = transforms.Compose([
    transforms.Resize((224,
                       
                        224)),
    transforms.ToTensor(),
])


In [6]:
# Prompt the user for the folder location
folder_url = input('Enter folder location: ')

In [7]:
# Load the dataset
dataset = ImageFolder(folder_url, transform=transform_img_normal)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=50, shuffle=False)

In [8]:
# Function to predict the class labels
def predict(model, data):
    class_labels = ['Bad Apple', 'Normal Apple', 'Rot Apple', 'Scab Apple']
    class_counts = [0, 0, 0, 0]

    with torch.no_grad():
        for inputs, labels in data:
            inputs = inputs.to(device)
            out = model(inputs)
            _, predicted = torch.max(out.data, 1)
            
            for p in predicted:
                class_counts[p.item()] += 1

    label_counts_dict = {label: count for label, count in zip(class_labels, class_counts)}
    return label_counts_dict

In [9]:
# Perform prediction on the dataset
apple_counts = predict(model, dataset_loader)
mylist = [apple_counts]


In [10]:
from sentence_transformers import SentenceTransformer, util

# Load the SentenceTransformer model
modelchat = SentenceTransformer('all-MiniLM-L12-v2')

# Create an empty list to store your passages
mylist = []
mylist.append("AQL(Acceptable Quality Limit) of class 1 is 0.4 or less than 0.4 and it can be accepted as class 1 if n(batch size)=32 and x(the number of not Normal Apple) < 1.")
mylist.append("AQL of class 2 is more than 0.4 and less than 6.5 or equal to 6.5 and it can be accepted as class 2 if n=20 and x<8.")
mylist.append("AQL of class 3 is more than 6.5 and less than 15 or equal to 15 and it can be accepted as class 3 if n=20 and x<15.")
mylist.append("AQL of class 4 is more than 15 and it can be class 4 if n=20 and x >= 15.")
mylist.append("The quality of the batch is very good if it is class 1.")
mylist.append("The quality of the batch is bad if it is class 4.")
mylist.append("Only 'Normal Apple' can be accepted in AQL")

# Encode the passages
passage_embeddings = modelchat.encode(mylist)

# Prompt the user for a query
query = input('What is your question?')
#'What is the classification of this batch? '

# Encode the query
query_embedding = modelchat.encode([query])

# Calculate the similarity scores
similarity_scores = util.cos_sim(query_embedding, passage_embeddings)

# Find the index of the most similar passage
most_similar_index = similarity_scores.argmax()

# Retrieve the most similar passage and its similarity score
most_similar_passage = mylist[most_similar_index]
similarity_score = similarity_scores[0][most_similar_index]

# Print the most similar passage and its similarity score
print('Most similar passage:', most_similar_passage)
print('Similarity score:', similarity_score)

# Extract class of apples from the most similar passage
class_of_apples = most_similar_passage.split(' ')[-1]

# Extract AQL from the most similar passage
aql_start = most_similar_passage.find('AQL') + 4
aql_end = most_similar_passage.find('is') - 1
aql = most_similar_passage[aql_start:aql_end]

# Calculate the accuracy of the model
accuracy = similarity_score * 100

# Print the class of apples, AQL, and accuracy
print('Class of apples:', class_of_apples)
print('AQL:', aql)
print('Accuracy:', accuracy)
print("Similarity:", util.dot_score(query_embedding, passage_embeddings))


Most similar passage: Only 'Normal Apple' can be accepted in AQL
Similarity score: tensor(0.4279)
Class of apples: AQL
AQL: 
Accuracy: tensor(42.7904)
Similarity: tensor([[0.3749, 0.3471, 0.4024, 0.3555, 0.0348, 0.0886, 0.4279]])
